In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [ ]:
#read the sales csv file

df = pd.read_csv("Sales.csv", encoding='latin1')
df.isna().sum()

In [29]:
#change Dataframe
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Delivery Date"]= pd.to_datetime(df["Delivery Date"])

#finding shipping interval
df["shipping_interval"] = (df["Delivery Date"] - df["Order Date"]).dt.days
df["Delivery Date"] = df["Delivery Date"].fillna(df["Order Date"] + pd.Timedelta(days=4))
df["shipping_interval"] = (df["Delivery Date"] - df["Order Date"]).dt.days

In [34]:
#read the exchange file

df1=pd.read_csv("exchange_rates.csv")

# change datatype
df1["Date"]=pd.to_datetime(df1["Date"])

#mergge the data sales and exchange file
merged_df = pd.merge(df, df1, left_on=['Order Date', 'Currency Code'], right_on=['Date', 'Currency'], how='left')

#change column name
merged_df['USD'] = merged_df['Exchange']

#drop the unwanted columns
merged_df.drop(columns=["Date","Currency","Exchange","Currency Code"],inplace=True)



In [ ]:
#read the product file

df=pd.read_csv("products.csv")

#using regex remove the $ and type as float for calculation
df["Unit Cost USD"] = df["Unit Cost USD"].replace('[\$,]', '', regex=True)
df["Unit Price USD"] = df["Unit Price USD"].replace('[\$,]', '', regex=True)
df["Unit Cost USD"]=df["Unit Cost USD"].astype(float)
df["Unit Price USD"]=df["Unit Price USD"].astype(float)

#after merge the the above dataframe
merge=pd.merge(merged_df,df,left_on=['ProductKey'], right_on=["ProductKey"], how='left') 

#drop unwantwd columns because i have the names its enough
merge.drop(columns=["SubcategoryKey","CategoryKey"],inplace=True)

#calculation cost & revenue
merge["Cost_Price"]=merge["Unit Cost USD"]*merge["Quantity"]
merge["Revenue"]=merge["Unit Price USD"]*merge["Quantity"]*merge["USD"]

#drop unwantwd columns
merge.drop(columns=["Unit Price USD","Unit Cost USD","USD"],inplace=True)

#Profit=Selling Price−Cost Price
merge["profit"]=merge["Revenue"]-merge["Cost_Price"]

#Percentage of Profit=( Profit/cost profit)×100
merge["Profit_margin"]=(merge["profit"]/merge["Revenue"])*100

In [53]:
#store csv read

sl=pd.read_csv("stores.csv")

#change datatype
sl["Open Date"]=pd.to_datetime(sl["Open Date"])

#change column name 
sl["store_state"]=sl["State"]
sl["store_Country"]=sl["Country"]

#unwanted column
sl.drop(columns=["Square Meters","State","Country"],inplace=True)

#merge updated dataframe and store csv
merged_df=pd.merge(merge,sl,left_on=['StoreKey'], right_on=["StoreKey"], how='left')

In [56]:
#read the customer csv
cus=pd.read_csv("Customers.csv",encoding='latin1')
# change dataframe and column name
cus["Birthday"]=pd.to_datetime(cus["Birthday"])
cus["Customer_State"]=cus["State"]
cus["Customer_Country"]=cus["Country"]
cus.drop(columns=["State","Country","State Code"],inplace=True)

#merge
merged=pd.merge(merged_df,cus,left_on=['CustomerKey'], right_on=["CustomerKey"], how='left')

#find age of customer till date

reference_date = datetime.today()

# Calculate the age
merged['Age'] =(reference_date.year - merged['Birthday'].dt.year).astype(int)
merged.drop(columns=["Birthday"],inplace=True)

In [55]:
#needed view for full dataframe without hide
pd.set_option('display.max_columns', None)

In [78]:
#rearrage column name
ordered_columns = [
    'Order Number', 'Line Item', 'Order Date', 'Delivery Date','shipping_interval','CustomerKey', 'Name', 'Gender', 'Age',"Continent",
    'Customer_Country', 'Customer_State', 'City', 'Zip Code', 'StoreKey', 'store_Country', 'store_state', 
    'Open Date', 'Quantity', 'ProductKey', 'Product Name','Brand', 'Category', 'Subcategory', 'Color','Quantity' ,'Cost_Price', 'Revenue', 
    'profit','Profit_margin']

df = merged[ordered_columns]

In [ ]:
df

In [79]:
#connect mysql database in local

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Vasu@3003",
    database="eda"
)

cursor = conn.cursor()

# Define the SQL table schema
create_table_query = """
CREATE TABLE IF NOT EXISTS SALES_DATA (
    OrderNumber INT,
    LineItem INT,
    OrderDate DATE,
    DeliveryDate DATE,
    shipping_interval INT,
    CustomerKey INT,
    Name VARCHAR(255),
    Gender VARCHAR(10),
    Age INT,
    Continent VARCHAR(50),
    Customer_Country VARCHAR(100),
    Customer_State VARCHAR(100),
    City VARCHAR(100),
    Zip_Code VARCHAR(20),
    StoreKey INT,
    store_Country VARCHAR(100),
    store_state VARCHAR(100),
    Open_Date DATE,
    Quantity INT,
    ProductKey INT,
    Product_Name VARCHAR(255),
    Brand VARCHAR(100),
    Category VARCHAR(100),
    Subcategory VARCHAR(100),
    Color VARCHAR(50), 
    Cost_Price DECIMAL(10, 2),
    Revenue DECIMAL(10, 2),
    profit DECIMAL(10, 2),
    Profit_margin DECIMAL(10, 2)
)
"""
cursor.execute(create_table_query)
conn.commit()

In [80]:
# Insert data into the table
insert_query = """
INSERT INTO SALES_DATA (OrderNumber, LineItem, OrderDate, DeliveryDate, shipping_interval, CustomerKey, Name, Gender, Age, Continent,
    Customer_Country, Customer_State, City, `Zip_Code`, StoreKey, store_Country, store_state, Open_Date, Quantity, ProductKey,
    Product_Name, Brand, Category, Subcategory, Color, Cost_Price, Revenue, profit, Profit_margin)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


for index, row in df.iterrows():
    # Ensure the row values are in the correct order
    cursor.execute(insert_query, [row[column] for column in ordered_columns])

conn.commit()
